# Ingest knowledge base data t a Vector DB

In this notebook we take knowledge base data available in an S3 bucket and convert it into embeddings using an LLM (such as`GPT-J-6B`) and store it in a vector database. We use `SagemakerEndpointEmbeddings` class provided by [`langchain`](https://python.langchain.com/en/latest/index.html) to create the embeddings and store them as a [`FAISS`](https://github.com/facebookresearch/faiss) index. The [`FAISS`](https://github.com/facebookresearch/faiss) index is then uploaded to an S3 bucket from where it can be downloaded into an application that intends to retrieve matching documents (say for a question answer use-case).

In [2]:
!pip install --upgrade sagemaker --quiet
!pip install ipywidgets==7.0.0 --quiet
!pip install langchain==0.0.123
!pip install faiss-cpu --quiet

In [25]:
import os
import sys
import glob
import time
import logging
import pandas as pd
import sagemaker, boto3, json
from sagemaker.model import Model
from sagemaker.session import Session
from sagemaker.predictor import Predictor
from sagemaker.utils import name_from_base
from sagemaker import image_uris, model_uris, script_uris, hyperparameters

In [19]:
import json
from typing import Any, Dict, List, Optional
from langchain.embeddings import SagemakerEndpointEmbeddings
from langchain.llms.sagemaker_endpoint import ContentHandlerBase

In [39]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma, AtlasDB, FAISS
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.chains.question_answering import load_qa_chain

In [60]:
# global constants
APP_NAME = "qa-w-rag"
RAW_DATA_S3_PREFIX = "s3://jumpstart-cache-prod-us-east-2/training-datasets/Amazon_SageMaker_FAQs/"
DATA_DIR = "data"
RAW_DATA_DIR = os.path.join(DATA_DIR, "raw")
PROCESSED_DATA_DIR = os.path.join(DATA_DIR, "processed")
VECTOR_DB_DIR = os.path.join(DATA_DIR, "vectordb")
KB_FNAME = "kb.csv"
KB_DATAFRAME_COL_INDEXES_TO_KEEP = [1]

In [5]:
logger = logging.getLogger()
logging.basicConfig(format='%(asctime)s,%(module)s,%(processName)s,%(levelname)s,%(message)s', level=logging.INFO, stream=sys.stderr)

In [6]:
%store -r embedding_model_endpoint_name

In [7]:
logger.info(f"embedding_model_endpoint_name={embedding_model_endpoint_name}")

2023-04-13 01:07:18,525,1440690944,MainProcess,INFO,embedding_model_endpoint_name=qa-w-rag-huggingface-textembedding-gpt--2023-04-12-15-09-23-080


In [61]:
sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()
model_version = "*"
bucket = sagemaker_session.default_bucket()
logger.info(f"aws_role={aws_role}, aws_region={aws_region}, bucket={bucket}")

2023-04-13 02:02:40,868,1638410780,MainProcess,INFO,aws_role=arn:aws:iam::015469603702:role/SageMakerRepoRole, aws_region=us-east-1, bucket=sagemaker-us-east-1-015469603702


In [10]:
# extend the SagemakerEndpointEmbeddings class from langchain to provide a custom embedding function
class SagemakerEndpointEmbeddingsJumpStart(SagemakerEndpointEmbeddings):
    def embed_documents(
        self, texts: List[str], chunk_size: int = 5
    ) -> List[List[float]]:
        """Compute doc embeddings using a SageMaker Inference Endpoint.

        Args:
            texts: The list of texts to embed.
            chunk_size: The chunk size defines how many input texts will
                be grouped together as request. If None, will use the
                chunk size specified by the class.

        Returns:
            List of embeddings, one for each text.
        """
        results = []
        _chunk_size = len(texts) if chunk_size > len(texts) else chunk_size
        
        for i in range(0, len(texts), _chunk_size):
            response = self._embedding_func(texts[i : i + _chunk_size])
            results.extend(response)
        return results

# class for serializing/deserializing requests/responses to/from the embeddings model
class ContentHandler(ContentHandlerBase):
    content_type = "application/json"
    accepts = "application/json"
    def transform_input(self, prompt: str, model_kwargs = {}) -> bytes:
        input_str = json.dumps({"text_inputs": prompt, **model_kwargs})
        return input_str.encode('utf-8') 
    
    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        embeddings = response_json["embedding"]
        if len(embeddings) == 1:
            return embeddings[0]
        return embeddings

# all set to create the objects for the ContentHandler and SagemakerEndpointEmbeddingsJumpStart classes
content_handler = ContentHandler()

# note the name of the LLM Sagemaker endpoint, this is the model that we would be using for generating the embeddings
embeddings = SagemakerEndpointEmbeddingsJumpStart( 
    endpoint_name=embedding_model_endpoint_name,
    region_name=aws_region, 
    content_handler=content_handler
)

---
## Download the data

The data is stored in an S3 bucket, we download the files from there and read it in a `langchain` document list.

In [21]:
!aws s3 cp --recursive $RAW_DATA_S3_PREFIX $RAW_DATA_DIR

download: s3://jumpstart-cache-prod-us-east-2/training-datasets/Amazon_SageMaker_FAQs/Amazon_SageMaker_FAQs.csv to data/raw/Amazon_SageMaker_FAQs.csv


There could be multiple files in the directory.

In [23]:
files = glob.glob(os.path.join(RAW_DATA_DIR, "*.csv"))
logger.info(f"raw data files = {files}")

2023-04-13 01:32:31,565,4131182362,MainProcess,INFO,raw data files = ['data/raw/Amazon_SageMaker_FAQs.csv']


In [31]:
df_kb_list = []
for f in files:
    df_kb = pd.read_csv(f, header=None)
    logger.info(f"read {f}, dataframe shape={df_kb.shape}")
    df_kb_list.append(df_kb)
df_kb = pd.concat(df_kb_list)
logger.info(f"shape of dataframe containing all raw data = {df_kb.shape}")
    

2023-04-13 01:36:49,316,149979566,MainProcess,INFO,read data/raw/Amazon_SageMaker_FAQs.csv, dataframe shape=(154, 2)
2023-04-13 01:36:49,317,149979566,MainProcess,INFO,shape of dataframe containing all raw data = (154, 2)


In [32]:
display(df_kb.head())

,0,1
0,What is Amazon SageMaker?,Amazon SageMaker is a fully managed service to...
1,In which Regions is Amazon SageMaker available...,For a list of the supported Amazon SageMaker A...
2,What is the service availability of Amazon Sag...,Amazon SageMaker is designed for high availabi...
3,How does Amazon SageMaker secure my code?,Amazon SageMaker stores code in ML storage vol...
4,What security measures does Amazon SageMaker h...,Amazon SageMaker ensures that ML model artifac...


In [38]:
df_kb = df_kb[KB_DATAFRAME_COL_INDEXES_TO_KEEP]
display(df_kb.head())

,1
0,Amazon SageMaker is a fully managed service to...
1,For a list of the supported Amazon SageMaker A...
2,Amazon SageMaker is designed for high availabi...
3,Amazon SageMaker stores code in ML storage vol...
4,Amazon SageMaker ensures that ML model artifac...


---

## Load the data in a `FAISS` index

In [45]:
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)
processed_data_fpath = os.path.join(PROCESSED_DATA_DIR, KB_FNAME)
logger.info(f"writing processed data, dataframe shape = {df_kb.shape} to {processed_data_fpath}")
df_kb.to_csv(fpath, index=False)

2023-04-13 01:44:05,371,4124703,MainProcess,INFO,writing processed data, dataframe shape = (154, 1) to data/processed/kb.csv


Now load this data into `langchain` `documents`.

In [46]:
loader = CSVLoader(file_path=processed_data_fpath)

In [47]:
documents = loader.load()

In [50]:
# just print first n documents
n = 5
for doc in documents[:n]:
    logger.info(f"---------------")
    logger.info(doc)

2023-04-13 01:46:58,214,3674270965,MainProcess,INFO,---------------
2023-04-13 01:46:58,216,3674270965,MainProcess,INFO,page_content='1: Amazon SageMaker is a fully managed service to prepare data and build, train, and deploy machine learning (ML) models for any use case with fully managed infrastructure, tools, and workflows.' lookup_str='' metadata={'source': 'data/processed/kb.csv', 'row': 0} lookup_index=0
2023-04-13 01:46:58,216,3674270965,MainProcess,INFO,---------------
2023-04-13 01:46:58,217,3674270965,MainProcess,INFO,page_content='1: For a list of the supported Amazon SageMaker AWS Regions, please visit the\xa0AWS Regional Services page. Also, for more information, see\xa0Regional endpoints\xa0in the AWS general reference guide.' lookup_str='' metadata={'source': 'data/processed/kb.csv', 'row': 1} lookup_index=0
2023-04-13 01:46:58,218,3674270965,MainProcess,INFO,---------------
2023-04-13 01:46:58,219,3674270965,MainProcess,INFO,page_content='1: Amazon SageMaker is designed

Now, we can build the QA application. Firstly, we **generate embedings for each of document in the knowledge library with SageMaker GPT-J-6B embedding model.**

In [51]:
vectordb = FAISS.from_documents(documents, embeddings)

2023-04-13 01:48:53,548,loader,MainProcess,INFO,Loading faiss with AVX2 support.
2023-04-13 01:48:53,568,loader,MainProcess,INFO,Successfully loaded faiss with AVX2 support.


Save the FAISS index locally so that it can be uploaded to S3

In [53]:
os.makedirs(VECTOR_DB_DIR, exist_ok=True)

In [54]:
vectordb.save_local(VECTOR_DB_DIR)

---

## Upload the Vector DB to S3

In [63]:
!ls -ltr $VECTOR_DB_DIR
!aws s3 cp $VECTOR_DB_DIR s3://$bucket/$APP_NAME/vectordb --recursive

total 2552
-rw-r--r-- 1 sagemaker-studio root 2523181 Apr 13 01:58 index.faiss
-rw-r--r-- 1 sagemaker-studio root   84216 Apr 13 01:58 index.pkl
upload: data/vectordb/index.pkl to s3://sagemaker-us-east-1-015469603702/qa-w-rag/vectordb/index.pkl
upload: data/vectordb/index.faiss to s3://sagemaker-us-east-1-015469603702/qa-w-rag/vectordb/index.faiss
